Das Notebook ist dafür da die User Stories auszuführen. Bitte testet sie davor in eurem .main (und macht euer main in das gitignore). Hier also nur funktionierende User Stories!

Damit der import richtig funktioniert müsst ihr die init datei des jeweiligen Ordners (model/data_access/business_logic) pflegen.

In [ ]:
import os
import shutil

import business_logic.address_manager

original = "database/original.db"
working = "database/hotel_reservation_sample_working.db"
shutil.copyfile(original, working)



os.environ["DB_FILE"] = working

from datetime import date

import business_logic.room_manager
import model
import data_access
import business_logic
import ui

Hotel_manager = business_logic.HotelManager()
Room_manager = business_logic.RoomManager()
Booking_manager= business_logic.BookingManager()
Invoice_manager = business_logic.InvoiceManager()
Adress_manager = business_logic.AddressManager()



1. Als Gast möchte ich die verfügbaren Hotels durchsuchen, damit 
ich dasjenige auswählen kann, welches meinen Wünschen 
entspricht. Wünsche sind:

1.1. Ich möchte alle Hotels in einer Stadt durchsuchen, 
damit ich das Hotel nach meinem bevorzugten Standort 
(Stadt) auswählen kann.

In [ ]:
# UI
input_city = ui.hotel_ui.get_city_input()

# Business Logic
by_city = Hotel_manager.get_hotels_by_city(input_city)

#Output
if not by_city:
    print(f"No hotels found in: {input_city}.")
else:
    Hotel_manager.print_user_friendly_hotels(by_city)

Erklärung:

UI: In der Hotel UI wird durch get_city_input sichergestellt das mindestens 3 Buchstaben und höchstens 10 inputed werden. Der User könnte zwar integers eingeben, aber bekommt dann kein Ergebniss. Durch einen while loop wird es bei falschen input weiter versucht bis ein richtiger kommt.

BL: Es werden alle Hotels in die Business Logic (BL) geladen durch self._all_hotels . Dann wird gefiltert nach city.lower() in hotel.address.city.lower(). Dadurch muss man nicht die Stadt genau ausschreiben.

1.2. Ich möchte alle Hotels in einer Stadt nach der
Anzahl der Sterne (z.B. mindestens 4 Sterne) durchsuchen.

In [ ]:
# UI
input_city = ui.hotel_ui.get_city_input()
input_stars = ui.hotel_ui.get_stars_input()

# Business Logic
by_city_and_stars = Hotel_manager.get_hotels_by_city_and_stars(input_city,input_stars)

# Output
if not by_city_and_stars:
    print(f"No hotels found in: {input_city} with {input_stars} stars.")
else:
    Hotel_manager.print_user_friendly_hotels(by_city_and_stars)


Erklärung:

UI: get_city input ist gleich wie oben. get_stars_input prüft das es ein integer ist und zwischen 1-5.

BL: Genau wie die obere User Story wird alles in der BL gefiltert. Nur wird jetzt eine neue Funktion mit dem "and hotel.stars >= stars"  benutzt, damit die Sterne beim Hotel gleich groß oder größer sind als vom User gewünscht. 

1.3. Ich möchte alle Hotels in einer Stadt durchsuchen,
die Zimmer haben, die meiner Gästezahl entsprechen (nur 1
Zimmer pro Buchung).

In [ ]:
# UI
input_city = ui.hotel_ui.get_city_input()
input_max_guests = ui.hotel_ui.get_max_guests_input()

# Business Logic
by_max_guests = Hotel_manager.get_hotels_by_city_and_max_guests(input_city, input_max_guests)

# Output
if not by_max_guests:
    print(f"No hotels found in: {input_city} with max guests: {input_max_guests}.")
else:
    Hotel_manager.print_user_friendly_hotels(by_max_guests)

Erklärung: 

UI: get_city_input wie oben. get_max_guests prüft ob der input ein int und zwischen 1 und 50 ist. Wir haben mal definiert das es wohl keine Zimmer mit max guests über 50 gibt.

BL: Im Hotelmanager wird wie vorher nach stadt gesucht. Jetzt wird aber der Room_manager aufgerufen, zuerst mit der funktion "get_room_details_for_hotel(hotel.hotel_id)" also werden die rooms mit der hotel.id geholt und dann wird durch "[room for room in rooms if room.max_guests >= max_guests]" geprüft ob die max guest gleich oder größer sind. Die passenden Hotels werden dann returned. Ich habe extra nicht die Room infos mitgegeben, da ich es im Hotelmanager klären wollte und der GAst "alle Hotels in einer Stadt" durchsuchen will.

1.4. Ich möchte alle Hotels in einer Stadt durchsuchen,
die während meines Aufenthaltes ("von" (check_in_date)
und "bis" (check_out_date)) Zimmer zur Verfügung haben,
damit ich nur relevante Ergebnisse sehe.

In [ ]:
# UI
input_city = ui.hotel_ui.get_city_input()

# Business Logic
h= Booking_manager.read_av_rooms_city(input_city, date(2025,4,14), date(2025,4,11))

# Output
print(h)

Erklärung:

Anna du musst noch irgendwie checken das die Dates richtig sind, also nicht in der vergangenheit usw. Und der Output auch noch schöner

1.5. Ich möchte Wünsche kombinieren können, z.B. die
verfügbaren Zimmer zusammen mit meiner Gästezahl und der
mindest Anzahl Sterne.

In [ ]:
#UI
city = ui.input_helper.input_y_n("Do you want to book in a specific city? (Y or N)", )
if city:
    input_city = ui.hotel_ui.get_city_input()
else: 
    input_city = None

stars = ui.input_helper.input_y_n("Do you want to book a hotel with a specific number of stars? (Y or N)", )
if stars:
    input_stars = ui.hotel_ui.get_stars_input()
else:
    input_stars = None

max_guests = ui.input_helper.input_y_n("Do you want to book a hotel with a specific number of max guests? (Y or N)", )
if max_guests:
    input_max_guests = ui.hotel_ui.get_max_guests_input()
else:
    input_max_guests = None

check_in = ui.input_helper.input_y_n("Do you want to book a hotel for a specific check-in and check-out date? (Y or N)", )
if check_in:
    av= Booking_manager.read_av_rooms_city(input_city, date(2025,4,14), date(2025,4,11))

# Business Logic
result = Hotel_manager.get_users_individual_wishes(input_city,stars,max_guests, check_in)

# Output
if not result:
    print("No hotels found with your wishes.")
else:
    Hotel_manager.print_user_friendly_hotels(result)

Erklärung:

UI: In der UI wird mit input_y_n bei jedem wunsch erstmal gefragt ob sie es wollen, wenn nein dann ist die variabel auch None. Sonst kommt man zu den Funktionen von der voherigen Userstories.

BL: In der Business Logic wird die Funktion: get_users_individual_wishes benutzt. Jedes Parameter ist auf None, wenn es aber gegeben wird werden die voherigen Funktionen eingesetzt. Dadurch muss der User nicht zwingend einen Wunsch eingeben und kann ganz individuell die Wünsche kombinieren.

1.6. Ich möchte die folgenden Informationen pro Hotel
sehen: Name, Adresse, Anzahl der Sterne.

In [ ]:
# UI

# Business Logic
all_hotels = Hotel_manager.read_all_hotels()

# Output
Hotel_manager.print_user_friendly_hotels(all_hotels)


Erklärung:

Es werden einfach alle Hotels auf die BL geladen und user friendly geprinted mit den gewünschten Informationen.

2. Als Gast möchte ich Details zu verschiedenen Zimmertypen
(Single, Double, Suite usw.), die in einem Hotel verfügbar
sind, sehen, einschliesslich der maximalen Anzahl von Gästen
für dieses Zimmer, Beschreibung, Preis und Ausstattung, um eine
fundierte Entscheidung zu treffen.

2.1. Ich möchte die folgenden Informationen pro Zimmer 
sehen: Zimmertyp, max. Anzahl der Gäste, Beschreibung, 
Ausstattung, Preis pro Nacht und Gesamtpreis.

In [ ]:
#UI + Business Logic
input_city = ui.hotel_ui.get_city_no_limit()

valid = False
while not valid:
    hotels_by_city = Hotel_manager.get_hotels_by_city(input_city)
    if not hotels_by_city:
        print(f"No hotels found in: {input_city}. Please try again.")
        input_city = ui.hotel_ui.get_city_input()
    else:
        for i, hotel in enumerate(hotels_by_city, 1):
            print(f"{i}. {ui.hotel_ui.get_user_frendly_hotel_info_short(hotel)}")
        try:
            input_idx = ui.input_helper.input_valid_int("Select Hotel: ", 1, len(hotels_by_city)) - 1
            selected_hotel = hotels_by_city[input_idx]
            valid = True
        except ui.input_helper.OutOfRangeError as e:
            print(f"Invalid input, please select a number between 1 and {len(hotels_by_city)}.")
        except ValueError as e:
            print("Invalid input, please enter a valid number.")

# Output
print(f"{selected_hotel.name} has following rooms:")
rooms = Room_manager.get_room_info_user_friendly(selected_hotel.hotel_id)

# Output
print(rooms)

Erklärung:

UI: Ich habe extra eine neue get_city funktion gemacht, damit der User auch keine City eingeben muss und alle sehen kann. Danach starte ein while loop, in dem die ausgewählten hotels (die in der Stadt sind) aufgelistet werden. Dafür habe ich eine neue funktion gemacht extra um nur Name, Stars und city zu bekommen. Dann soll der User ein Hotel aussuchen. ICh arbeite mit dem Index damit er das richtige wählen kann, und mache es einfach -1 um den Index zu bekommen. OutOfRange ist extra so gestaltet das nur maximal die länge der Liste von Hotels ausgewählt werden kann. Designed wurde es mithilfe vom Coach Phillip.

BL: Nachdem das Hotel ausgesucht wurde, wird die funktion get_room_info_user_friendly die mit der hotel_id arbeitet aufgerufen. Am ende wird es einfach noch geprinted.

2.2. Ich möchte nur die verfügbaren Zimmer sehen, sofern 
ich meinen Aufenthalt (von – bis) spezifiziert habe

In [ ]:
av_rooms= Booking_manager.read_all_av_rooms_by_hotel(4, date(2025,9,11), date(2025, 9, 14))
print(av_rooms)

3. Als Admin des Buchungssystems möchte ich die Möglichkeit haben, 
Hotelinformationen zu pflegen, um aktuelle Informationen im 
System zu haben.

3.1. Ich möchte neue Hotels zum System hinzufügen

In [ ]:
# UI
new_name = ui.input_helper.input_valid_string("Enter new hotel name: ", 1, 50)
new_stars = ui.input_helper.input_valid_int("Enter new hotel stars: ", 1, 5)
new_adress = ui.input_helper.input_y_n("Do you want to create a new adress for the hotel? (Y or N)", )
if new_adress:
    new_street = ui.input_helper.input_valid_string("Enter new street: ", 1, 50)
    new_city = ui.input_helper.input_valid_string("Enter new city: ", 1, 20)
    new_zip = ui.input_helper.input_valid_string("Enter new zip code: ", 1, 20)
    new_adress = Adress_manager.create_new_address(new_street, new_city, new_zip)
else:
    all_adresses = Adress_manager.read_all_addresses()
    for i, address in enumerate(all_adresses, 1):
        print(f"{i}. {ui.hotel_ui.get_address_short(address)}")

    new_adress = None
    while new_adress is None:
        try:
            input_idx = ui.input_helper.input_valid_int("Select address: ", 1, len(all_adresses)) - 1
            new_adress = all_adresses[input_idx]
        except ui.input_helper.OutOfRangeError as e:
            print(f"Invalid input, please select a number between 1 and {len(all_adresses)}.")
        except ValueError as e:
            print("Invalid input, please enter a valid number.")


# Business Logic
Hotel_manager.create_new_hotel(new_name, new_stars, new_adress.address_id)

        
# Output
# Im data_access 

Erklärung:

UI: Der Name und Sterne werden im Input helpern gemacht mit Limits. Bei der Addresse kann man wählen ob man eine neue erstellen möchte: (Dort wird mit weitern input helpern gearbeitet) oder ob man eine bereits bestehende wählen will. Wie bei einer voherigen User Story arbeite ich wieder mit dem Index und einem while loop, bis die neue Addresse gewählt wurde. Mit der neuen Addresse ob neu gemacht oder ausgewählt wird dann ein neues Hotel erstellt. Im DAL ist geregelt das man immer sieht was gerade erstellt wurde und was nun alles in der DB ist.

BL: Create new Hotel wird von der BL gemacht. address = "self._address_data_access.read_address_by_id(address_id)" hilft um die address_id die als Parameter gebraucht wird um ein neues Hotel zu erstellen. Bedeutet bevor man ein neues Hotel erstellen kann muss man eine Addresse haben.

3.2. Ich möchte Hotels aus dem System entfernen


In [ ]:
# UI
all_hotels = Hotel_manager.read_all_hotels()
for i, hotel in enumerate(all_hotels, 1):
    print(f"{i}. {ui.hotel_ui.get_user_frendly_hotel_info_short(hotel)}")

found = False
while not found:
    try:
        input_idx = ui.input_helper.input_valid_int("Select hotel to delete: ", 1, len(all_hotels)) - 1
        selected_hotel = all_hotels[input_idx]
        found = True
    except ui.input_helper.OutOfRangeError as e:
        print(f"Invalid input, please select a number between 1 and {len(all_hotels)}.")
    except ValueError as e:
        print("Invalid input, please enter a valid number.")

# Business Logic
Hotel_manager.delete_hotel_by_id(selected_hotel.hotel_id)
# Output
#im Data_access

Erklärung:

UI: Der Admin bekommt alle Hotels angezeigt welche im moment in der DB sind. Über den index kann er eins auswählen.

BL: Die hotel.id des gewählen hotels wird genutzt um mit der funktion delete_hotel_by_id in dem DAL die DB zu verändern.

3.3. Ich möchte die Informationen bestimmter Hotels 
aktualisieren, z. B. den Namen, die Sterne usw.

In [ ]:
# UI
all_hotels = Hotel_manager.read_all_hotels()
for i, hotel in enumerate(all_hotels, 1):
    print(f"{i}. {ui.hotel_ui.get_user_frendly_hotel_info_short(hotel)}")

selected_hotel = None
while selected_hotel is None:
    try:
        input_idx = ui.input_helper.input_valid_int("Select hotel to update: ", 1, len(all_hotels)) - 1
        selected_hotel = all_hotels[input_idx]
    except ui.input_helper.OutOfRangeError as e:
        print(f"Invalid input, please select a number between 1 and {len(all_hotels)}.")
    except ValueError as e:
        print("Invalid input, please enter a valid number.")

print(f"You selected: {ui.hotel_ui.get_user_frendly_hotel_info_short(selected_hotel)}")

new_name = selected_hotel.name
if ui.input_helper.input_y_n("Do you want to update the hotel name? (Y or N)", ):
    new_name = ui.input_helper.input_valid_string("Enter new hotel name: ", 1, 50)

new_stars = selected_hotel.stars
if ui.input_helper.input_y_n("Do you want to update the hotel stars? (Y or N)", ):
    new_stars = ui.input_helper.input_valid_int("Enter new hotel stars: ", 1, 5)

new_adress = selected_hotel.address
if ui.input_helper.input_y_n("Do you want to update the hotel address? (Y or N)", ):
    new_street = ui.input_helper.input_valid_string("Enter new street: ", 1, 50)
    new_city = ui.input_helper.input_valid_string("Enter new city: ", 1, 20)
    new_zip = ui.input_helper.input_valid_string("Enter new zip code: ", 1, 20)
    new_adress = Adress_manager.create_new_address(new_street, new_city, new_zip)
else:
    all_adresses = Adress_manager.read_all_addresses()
    for i, address in enumerate(all_adresses, 1):
        print(f"{i}. {ui.hotel_ui.get_address_short(address)}")

    new_adress = None
    while new_adress is None:
        try:
            input_idx = ui.input_helper.input_valid_int("Select address: ", 1, len(all_adresses)) - 1
            new_adress = all_adresses[input_idx]
        except ui.input_helper.OutOfRangeError as e:
            print(f"Invalid input, please select a number between 1 and {len(all_adresses)}.")
        except ValueError as e:
            print("Invalid input, please enter a valid number.")
    
# Business Logic
Hotel_manager.update_hotel(selected_hotel.hotel_id, new_name, new_stars, new_adress.address_id)

# Output
#im data_access

Erklärung:

UI: Erst muss der Admin ein Hotel wählen, wieder über den Index. Danach wird angezeigt welches Hotel er gewählt hat und er wird gefragt über input_y_n was er ändern will, ob name, stars oder addresse. Wenn er die Addresse ändern will, kann er entweder eine neue erstellen, wie wenn er ein neues Hotel erstellt, oder er kann eine aus der DB wählen und diese mit dem Hotel verknüpfen.

BL: Update hotel ist in der BL auf None gesetzt, also die parameter. Im DAL wird dann mit den daten ein neues Hotel Objekt erstellt, welches zu einer anderen FUnktion gegeben wird welches das objekt in die DB einträgt. Ich habe es so gemacht, weil die struktur schon am anfang so war und ich es charmant fand. (kreative Freiheit)

4. Als Gast möchte ich ein Zimmer in einem bestimmten Hotel 
buchen, um meinen Urlaub zu planen.

In [ ]:
# #Hotel nach Namen suchen
# hotels = Hotel_manager.read_hotel_by_name("National")
# print(hotels)

# #Available Rooms suchen
# av_rooms= Booking_manager.read_all_av_rooms_by_hotel_id(4, date(2025,9,11), date(2025, 9, 14))
# print(av_rooms)

# create booking aufrufen
booking= Booking_manager.create_new_booking(5, date(2025,5,11), date(2025, 5, 14), 2)
Booking_manager.read_all_bookings()


5. Als Gast möchte ich nach meinem Aufenthalt eine Rechnung 
erhalten, damit ich einen Zahlungsnachweis habe.
Hint: Fügt einen Eintrag in der «Invoice» Tabelle hinzu.

In [ ]:
Booking_manager.billing()
inv= Invoice_manager.read_all_invoice()
print(inv)

6. Als Gast möchte ich meine Buchung stornieren, damit ich nicht 
belastet werde, wenn ich das Zimmer nicht mehr benötige.
Hint: Sorgt für die entsprechende Invoice. 

In [ ]:
Booking_manager.cancell_booking(3)
bo= Booking_manager.read_booking_by_id(3)

7. Als Gast möchte ich eine dynamische Preisgestaltung auf der 
Grundlage der Nachfrage sehen, damit ich ein Zimmer zum besten 
Preis buchen kann.
Hint: Wendet in der Hochsaison höhere und in der Nebensaison 
niedrigere Tarife an.

In [ ]:
r = Booking_manager.read_av_rooms(date(2026,4,14), date(2026,4,11))
print(r)

8. Als Admin des Buchungssystems möchte ich alle Buchungen aller 
Hotels sehen können, um eine Übersicht zu erhalten.

In [ ]:
t=Booking_manager.read_all_bookings()
print(t)

9. Als Admin möchte ich eine Liste der Zimmer mit ihrer 
Ausstattung sehen, damit ich sie besser bewerben kann.


In [ ]:
print(Room_manager.get_rooms_for_admin())


10. Als Admin möchte ich in der Lage sein, Stammdaten zu verwalten, 
z.B. Zimmertypen, Einrichtungen, und Preise in Echtzeit zu 
aktualisieren, damit das Backend-System aktuelle Informationen 
hat. Hint: Stammdaten sind alle Daten, die nicht von anderen Daten 
abhängen.

In [ ]:
#new_room_type = Room_manager.create_new_room_type("Test", 100)

#all = Room_manager.read_all_room_types()
#print(all)